# TITLE

In [ ]:
import pandas as pd
import numpy as np
import statistics as stats

## Explore Electric Vehicle Data for the State of Washington

In [25]:
electric_vehicle_df = pd.read_csv("./data/Electric_Vehicle_Population_Data.csv")

In [26]:
electric_vehicle_df.head()

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,KM8K33AGXL,King,Seattle,WA,98103.0,2020,HYUNDAI,KONA,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,258,0,43.0,249675142,POINT (-122.34301 47.659185),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
1,1C4RJYB61N,King,Bothell,WA,98011.0,2022,JEEP,GRAND CHEROKEE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,1.0,233928502,POINT (-122.20578 47.762405),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
2,1C4RJYD61P,Yakima,Yakima,WA,98908.0,2023,JEEP,GRAND CHEROKEE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,14.0,229675939,POINT (-120.6027202 46.5965625),PACIFICORP,5.307700e+10
3,5YJ3E1EA7J,King,Kirkland,WA,98034.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,45.0,104714466,POINT (-122.209285 47.71124),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
4,WBY7Z8C5XJ,Thurston,Olympia,WA,98501.0,2018,BMW,I3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,97,0,22.0,185498386,POINT (-122.89692 47.043535),PUGET SOUND ENERGY INC,5.306701e+10


In [27]:
#Explore
print('Shape\n', electric_vehicle_df.shape)
print('\n')
print("Duplicates\n", electric_vehicle_df.duplicated().sum())
print('\n')
print('Null\n', electric_vehicle_df.isnull().sum())
print('\n')
print('Describe\n', electric_vehicle_df.describe())
print('\n')
print('Columns\n', electric_vehicle_df.columns)
print('\n')
print('Correlation\n', electric_vehicle_df.corr())

Shape
 (150482, 17)


Duplicates
 0


Null
 VIN (1-10)                                             0
County                                                 3
City                                                   3
State                                                  0
Postal Code                                            3
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                 341
DOL Vehicle ID                                         0
Vehicle Location                                       7
Electric Utility                                       3
2020 Census Tract                           

C:\Users\riley\AppData\Local\Temp\ipykernel_4424\1940688783.py:13: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print('Correlation\n', electric_vehicle_df.corr())


As we can see there are many 0 values throughout columns such as base MSRP and Electric Range, therefore, we will discard these columns as they are not relevant to our research. Additionally, location information is captured in several locations such as county, postal code, vehicle location, and 2020 Census Tract. We should only consider County from income data set as that is the only way to join the two.

We will begin first by dropping columns that have majority of 0 data and hard to work with location data. Then we will verify that all car data is from Washington.

In [28]:
electric_vehicle_df.drop(columns=["Electric Range", "Base MSRP", "Vehicle Location", "2020 Census Tract"], inplace=True)

In [29]:
print(electric_vehicle_df.loc[electric_vehicle_df['State'] != "WA"].count())

VIN (1-10)                                           341
County                                               338
City                                                 338
State                                                341
Postal Code                                          338
Model Year                                           341
Make                                                 341
Model                                                341
Electric Vehicle Type                                341
Clean Alternative Fuel Vehicle (CAFV) Eligibility    341
Legislative District                                   0
DOL Vehicle ID                                       341
Electric Utility                                     338
dtype: int64


In [30]:
print(electric_vehicle_df['Legislative District'].isnull().sum())

341


In [31]:
electric_vehicle_df = electric_vehicle_df.loc[electric_vehicle_df['State'] == "WA"]

In [32]:
print(electric_vehicle_df['Legislative District'].isnull().sum())

0


##### TODO FOR VEHICLE DF
Remove Noisy Location Data

Outliers

T Score check

Scaling

## Explore Washington Income Data

In [63]:
income_df = pd.read_csv("./data/median_household_income_estimates.csv",  thousands=',')

In [64]:
income_df.head()

,Unnamed: 0,1989,1990,1991,1992,1993,1994,1995,1996,1997,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Washington,31183.0,33417.0,34379.0,35882.0,36679.0,37895.0,38997.0,40568.0,42399.0,...,57284.0,60153.0,63439.0,65500.0,69288.0,72297.0,78674.0,80319.0,84155.0,86343.0
2,Adams,24604.0,28024.0,27301.0,28328.0,31277.0,29537.0,29604.0,31806.0,31795.0,...,43541.0,45712.0,47646.0,49501.0,48849.0,52870.0,53535.0,56421.0,57405.0,58406.0
3,Asotin,22897.0,24206.0,25110.0,26251.0,27359.0,27873.0,28288.0,29813.0,31499.0,...,42174.0,43368.0,46107.0,47020.0,51767.0,50746.0,54776.0,53377.0,56438.0,59675.0
4,Benton,32593.0,35012.0,37093.0,39228.0,41153.0,42790.0,42540.0,42817.0,44057.0,...,63710.0,63157.0,62071.0,62282.0,63502.0,67912.0,72847.0,75882.0,77339.0,78824.0


In [65]:
#Change First column name
income_df.rename(columns={'Unnamed: 0':'County'}, inplace=True)

#Drop First row of Nan Values
income_df = income_df.iloc[1:]

In [66]:
#Explore
print('Shape\n', income_df.shape)
print('\n')
print("Duplicates\n", income_df.duplicated().sum())
print('\n')
print('Null\n', income_df.isnull().sum())
print('\n')
print('Describe\n', income_df.describe())
print('\n')
print('Columns\n', income_df.columns)
print('\n')
print('Correlation\n', income_df.corr())

Shape
 (40, 35)


Duplicates
 0


Null
 County    0
1989      0
1990      0
1991      0
1992      0
1993      0
1994      0
1995      0
1996      0
1997      0
1998      0
1999      0
2000      0
2001      0
2002      0
2003      0
2004      0
2005      0
2006      0
2007      0
2008      0
2009      0
2010      0
2011      0
2012      0
2013      0
2014      0
2015      0
2016      0
2017      0
2018      0
2019      0
2020      0
2021      0
2022      0
dtype: int64


Describe
                1989          1990          1991          1992          1993  \
count     40.000000     40.000000     40.000000     40.000000     40.000000   
mean   26374.550000  28223.725000  28954.125000  30205.675000  31392.025000   
std     4189.680852   4575.695519   4753.131595   4889.560986   4807.684322   
min    20029.000000  20580.000000  21596.000000  22449.000000  23372.000000   
25%    23469.500000  25260.500000  25765.250000  27062.750000  28112.000000   
50%    25183.500000  27333.000000  27726.

C:\Users\riley\AppData\Local\Temp\ipykernel_4424\1478213490.py:13: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print('Correlation\n', income_df.corr())
